In [1]:
import pandas as pd
import numpy as np
import copy #複製 list 避免後續操作影響到原本的 list
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
# sklearn 中 preprocessing 的 Normalization套件  MinMaxScaler, StandardScaler 
from sklearn.model_selection import cross_val_score # cross validation score
from sklearn.linear_model import LinearRegression # 以線性回歸model驗證

In [2]:
# 讀取資料
data_path = 'data/'
df_train = pd.read_csv(data_path + 'house_train.csv.gz')
df_test = pd.read_csv(data_path + 'house_test.csv.gz')

# 重組資料成為訓練/預測用格式
train_Y = np.log1p(df_train['SalePrice'])
#  數據預處理時首先可以對偏度比較大的數據用og1p函數進行轉化，
# 使其更加服從高斯分佈，此步處理可能會使我們後續的分類結果得到一個好的結果
# log1p( )能夠避免复值得問題— 复值指一個自變量對應多個因變量
# log1p( ) 的使用就像是一個數據壓縮到了一個區間，與數據的標準類似
ids = df_test['Id']
# 為了將df_train和df_test合併起來，需要捨棄部分column
df_train = df_train.drop(['Id', 'SalePrice'], axis=1) #axis = 1 縱向操作，指整排column
df_test = df_test.drop(['Id'], axis=1)
df = pd.concat([df_train,df_test])


In [3]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


In [4]:
df_test

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [5]:
# 檢查缺失值

# isnull():檢查空值，回傳布林值
# notnull():檢查不是空值，回傳布林值
# dropna():刪除空值
# fillna():填入空值
df.isnull().sum().sort_values(ascending=False)

PoolQC           2909
MiscFeature      2814
Alley            2721
Fence            2348
FireplaceQu      1420
                 ... 
1stFlrSF            0
2ndFlrSF            0
LowQualFinSF        0
GrLivArea           0
SaleCondition       0
Length: 79, dtype: int64

In [6]:
# 對於 int64,float64 這兩種數值型欄位做整理，並存在 num_features中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'int64' or dtype == 'float64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features: {num_features}\n')

36 Numeric Features: ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']



In [7]:
print(df.dtypes)
print(df.columns)

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 79, dtype: object
Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFu

In [8]:
# 只留下df中數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
print(train_num)
df

1460


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,...,548.0,0,61,0,0,0,0,0,2,2008
1,20,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,...,460.0,298,0,0,0,0,0,0,5,2007
2,60,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,...,608.0,0,42,0,0,0,0,0,9,2008
3,70,60.0,9550,7,5,1915,1970,0.0,216.0,0.0,...,642.0,0,35,272,0,0,0,0,2,2006
4,60,84.0,14260,8,5,2000,2000,350.0,655.0,0.0,...,836.0,192,84,0,0,0,0,0,12,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,21.0,1936,4,7,1970,1970,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,6,2006
1455,160,21.0,1894,4,5,1970,1970,0.0,252.0,0.0,...,286.0,0,24,0,0,0,0,0,4,2006
1456,20,160.0,20000,5,7,1960,1996,0.0,1224.0,0.0,...,576.0,474,0,0,0,0,0,0,9,2006
1457,85,62.0,10441,5,5,1992,1992,0.0,337.0,0.0,...,0.0,80,32,0,0,0,0,700,7,2006


In [9]:
# 空缺值以 -1, 做線性回歸
df_m1 = df.fillna(-1) #把原本的 df 缺失值填入 -1，疊代為df_m1
# print(df_m1)
# print('='*50)
train_X = df_m1[:train_num]
print(df_m1[:train_num].shape)
print('='*50)
print(df_m1.shape)
estimator = LinearRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

(1460, 36)
(2919, 36)


0.8466400643386514

In [10]:
# 空缺值以 0, 做線性回歸
df_0 = df.fillna(0)
train_X = df_0[:train_num]
estimator = LinearRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.8466118155868807

In [11]:
# 空缺值以平均值, 做線性回歸
df_mn = df.fillna(df.mean())
train_X = df_mn[:train_num]
estimator = LinearRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.8442642432201248

In [12]:
# 空值補 -1，搭配最大最小
df = df.fillna(-1)
df_temp = MinMaxScaler().fit_transform(df)
train_X = df_temp[:train_num]
estimator = LinearRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.8461963829383949

In [13]:
# 搭配標準化
df_temp = StandardScaler().fit_transform(df)
train_X = df_temp[:train_num]
estimator = LinearRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.8465907391922668

In [14]:
df_f = df[num_features]
df_f
df_f.isnull().sum().sort_values()
df_f_1 = df_f['LotFrontage']
print(df_f_1.values.tolist())
df_f_2 = df_mn['LotFrontage']
print('==='*60)
print(df_f_2.values.tolist())

[65.0, 80.0, 68.0, 60.0, 84.0, 85.0, 75.0, -1.0, 51.0, 50.0, 70.0, 85.0, -1.0, 91.0, -1.0, 51.0, -1.0, 72.0, 66.0, 70.0, 101.0, 57.0, 75.0, 44.0, -1.0, 110.0, 60.0, 98.0, 47.0, 60.0, 50.0, -1.0, 85.0, 70.0, 60.0, 108.0, 112.0, 74.0, 68.0, 65.0, 84.0, 115.0, -1.0, -1.0, 70.0, 61.0, 48.0, 84.0, 33.0, 66.0, -1.0, 52.0, 110.0, 68.0, 60.0, 100.0, 24.0, 89.0, 66.0, 60.0, 63.0, 60.0, 44.0, 50.0, -1.0, 76.0, -1.0, 72.0, 47.0, 81.0, 95.0, 69.0, 74.0, 85.0, 60.0, 21.0, -1.0, 50.0, 72.0, 60.0, 100.0, 32.0, 78.0, 80.0, -1.0, 121.0, 122.0, 40.0, 105.0, 60.0, 60.0, 85.0, 80.0, 60.0, 69.0, -1.0, 78.0, 73.0, 85.0, 77.0, -1.0, 77.0, 64.0, 94.0, -1.0, 75.0, 60.0, 50.0, 85.0, 105.0, 75.0, -1.0, 77.0, -1.0, 61.0, 34.0, -1.0, 74.0, 90.0, 65.0, -1.0, 50.0, 75.0, 55.0, 48.0, 60.0, -1.0, 55.0, 69.0, 69.0, 88.0, -1.0, 75.0, -1.0, 78.0, 80.0, -1.0, 82.0, 73.0, 65.0, 70.0, 78.0, 71.0, 78.0, 70.0, 24.0, 51.0, -1.0, 63.0, -1.0, 120.0, 107.0, -1.0, -1.0, 84.0, 60.0, 60.0, 92.0, 100.0, 134.0, -1.0, 110.0, 95.0, 55.0